CONECTO CON MI DRIVE

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#Pongo la ruta a mi carpeta de TFM y compruebo que están los datos que necesito
import os
BASE = "/content/drive/MyDrive/MASTER GML/TFM"

DIR_GOLD = os.path.join(BASE, "cronicas_gold_standard")
DIR_LLM  = os.path.join(BASE, "cronicas_limpias")

print("Ruta GOLD:", DIR_GOLD)
print("Existe GOLD:", os.path.exists(DIR_GOLD))

print("Ruta LLM:", DIR_LLM)
print("Existe LLM:", os.path.exists(DIR_LLM))

if os.path.exists(DIR_GOLD):
    print("TXT humanos:", len([f for f in os.listdir(DIR_GOLD) if f.lower().endswith(".txt")]))

if os.path.exists(DIR_LLM):
    print("TXT LLM:", len([f for f in os.listdir(DIR_LLM) if f.lower().endswith(".txt")]))


Ruta GOLD: /content/drive/MyDrive/MASTER GML/TFM/cronicas_gold_standard
Existe GOLD: True
Ruta LLM: /content/drive/MyDrive/MASTER GML/TFM/cronicas_limpias
Existe LLM: True
TXT humanos: 4
TXT LLM: 144


In [5]:
#Creo el dataframe con los textos humanos y los generados con LM
import pandas as pd

def leer_txt(path):
    with open(path, "r", encoding="utf-8", errors="replace") as f:
        return f.read().strip()

# Cargar crónicas humanas
rows = []

for fname in os.listdir(DIR_GOLD):
    if fname.lower().endswith(".txt"):
        texto = leer_txt(os.path.join(DIR_GOLD, fname))
        rows.append({
            "id": fname.replace(".txt", ""),
            "origen": "humano",
            "texto": texto
        })

# Cargar crónicas LLM
for fname in os.listdir(DIR_LLM):
    if fname.lower().endswith(".txt"):
        texto = leer_txt(os.path.join(DIR_LLM, fname))
        rows.append({
            "id": fname.replace(".txt", ""),
            "origen": "llm",
            "texto": texto
        })

df = pd.DataFrame(rows)

print("Total textos:", len(df))
print(df["origen"].value_counts())
df.head()


Total textos: 148
origen
llm       144
humano      4
Name: count, dtype: int64


,id,origen,texto
0,gold_f1_Dortmund__Real_Madrid,humano,Carvajal sirve la 15 al Rey de Europa\nUn cabe...
1,gold_f2_Bayern_Munich___Dinamo_Zagreb,humano,El Bayern hace historia con nueve goles al Din...
2,gold_f3_Liverpool___Milán,humano,El Liverpool despierta fantasmas en San Siro\n...
3,gold_f4_PSG___Arsenal,humano,Arteta y su Arsenal dan un golpe encima de la ...
4,gemma_3_4b_f1_c01,llm,El Real Madrid se corona en Wembley y destroza...


In [6]:
#Vamos a extraer modelo, ficha y configuración

import re
import numpy as np

# patrón: (modelo)_f(n)_c(nn)
pat = re.compile(r"^(?P<modelo>.+?)_f(?P<ficha>\d+)_c(?P<config>\d+)$", re.IGNORECASE)

def extraer_meta(row):
    if row["origen"] != "llm":
        return pd.Series({"modelo": "humano", "ficha": np.nan, "config": np.nan})
    m = pat.match(row["id"])
    if not m:
        return pd.Series({"modelo": None, "ficha": None, "config": None})
    return pd.Series({
        "modelo": m.group("modelo"),
        "ficha": int(m.group("ficha")),
        "config": int(m.group("config")),
    })

df[["modelo","ficha","config"]] = df.apply(extraer_meta, axis=1)

# comprobaciones
print("LLM sin parsear (debería ser 0):", df[(df.origen=="llm") & (df.modelo.isna())].shape[0])
print(df.groupby(["origen","modelo"]).size().head(10))

df.head(8)


LLM sin parsear (debería ser 0): 0
origen  modelo        
humano  humano             4
llm     gemma_3_4b        48
        mistral           48
        qwen_3_4b_2507    48
dtype: int64


,id,origen,texto,modelo,ficha,config
0,gold_f1_Dortmund__Real_Madrid,humano,Carvajal sirve la 15 al Rey de Europa\nUn cabe...,humano,NaN,NaN
1,gold_f2_Bayern_Munich___Dinamo_Zagreb,humano,El Bayern hace historia con nueve goles al Din...,humano,NaN,NaN
2,gold_f3_Liverpool___Milán,humano,El Liverpool despierta fantasmas en San Siro\n...,humano,NaN,NaN
3,gold_f4_PSG___Arsenal,humano,Arteta y su Arsenal dan un golpe encima de la ...,humano,NaN,NaN
4,gemma_3_4b_f1_c01,llm,El Real Madrid se corona en Wembley y destroza...,gemma_3_4b,1.0,1.0
5,gemma_3_4b_f1_c02,llm,El Real Madrid se corona en Wembley y destroza...,gemma_3_4b,1.0,2.0
6,gemma_3_4b_f1_c03,llm,El Real Madrid se corona en Wembley tras un fi...,gemma_3_4b,1.0,3.0
7,gemma_3_4b_f1_c04,llm,El Real Madrid se corona en Wembley y silencia...,gemma_3_4b,1.0,4.0


In [7]:
#Instalo librerías para tokenizar, lematiza, etc
!pip install -q spacy lexical-diversity
!python -m spacy download es_core_news_sm -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 53.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [8]:
import spacy
from lexical_diversity import lex_div as ld
nlp = spacy.load("es_core_news_sm")
print("spaCy cargado correctamente")


spaCy cargado correctamente


In [9]:
# Hago el proceso con un texto de ejemplo, para ver cómo funciona
# cogemos un texto humano de ejemplo
texto_ejemplo = df[df["origen"] == "humano"].iloc[0]["texto"]

print(texto_ejemplo[:500])  # mostramos solo el inicio


Carvajal sirve la 15 al Rey de Europa
Un cabezazo soberbio del canterano inclina una final igualadísima, con un Dortmund pujante frenado por Courtois en el primer tiempo. Vinicius cierra la victoria en el tramo final. La Champions es esa competición que todos quieren, pero que domina el Real Madrid. El equipo blanco no la juega, la gana. Después de sufrir y ser inferiores al Dortmund en un primer tiempo sorprendente, el Madrid golpeó de manera inesperada, en un balón aéreo, tras un córner de Kro


In [10]:
doc = nlp(texto_ejemplo)

print("Número total de tokens:", len(doc))
print("Número de oraciones:", len(list(doc.sents)))


Número total de tokens: 1198
Número de oraciones: 68


In [11]:
for token in doc[:20]:
    print(
        f"TEXTO: {token.text:15} | LEMA: {token.lemma_:15} | POS: {token.pos_}"
    )


TEXTO: Carvajal        | LEMA: Carvajal        | POS: PROPN
TEXTO: sirve           | LEMA: servir          | POS: VERB
TEXTO: la              | LEMA: el              | POS: DET
TEXTO: 15              | LEMA: 15              | POS: NUM
TEXTO: al              | LEMA: al              | POS: ADP
TEXTO: Rey             | LEMA: Rey             | POS: PROPN
TEXTO: de              | LEMA: de              | POS: ADP
TEXTO: Europa          | LEMA: Europa          | POS: PROPN
TEXTO: 
               | LEMA: 
               | POS: SPACE
TEXTO: Un              | LEMA: uno             | POS: DET
TEXTO: cabezazo        | LEMA: cabezazo        | POS: NOUN
TEXTO: soberbio        | LEMA: soberbio        | POS: ADJ
TEXTO: del             | LEMA: del             | POS: ADP
TEXTO: canterano       | LEMA: canterano       | POS: NOUN
TEXTO: inclina         | LEMA: inclín          | POS: ADJ
TEXTO: una             | LEMA: uno             | POS: DET
TEXTO: final           | LEMA: final           | POS: NOUN
TE

In [12]:
for i, sent in enumerate(doc.sents):
    print(f"\nORACIÓN {i+1}:")
    print(sent.text)
    if i == 2:
        break



ORACIÓN 1:
Carvajal sirve la 15 al Rey de Europa
Un cabezazo soberbio del canterano inclina una final igualadísima, con un Dortmund pujante frenado por Courtois en el primer tiempo.

ORACIÓN 2:
Vinicius cierra la victoria en el tramo final.

ORACIÓN 3:
La Champions es esa competición que todos quieren, pero que domina el Real Madrid.


In [13]:
# Calcular para cada crónica: n_words = número de “palabras” según nuestra definición:solo tokens alfabéticos (is_alpha),incluye nombres propios y acrónimos, excluye números en dígitos y puntuación
def contar_palabras(texto):
    doc = nlp(texto)
    palabras = [t for t in doc if t.is_alpha]  # nuestra definición
    return len(palabras)

# calcular n_words
df["n_words"] = df["texto"].apply(contar_palabras)

# resumen rápido
print("Resumen por origen (n_words):")
print(df.groupby("origen")["n_words"].describe())

# ver las más cortas y más largas (por si hay algo raro)
print("\n5 más cortas:")
display(df.sort_values("n_words").head(5)[["id","origen","n_words"]])

print("\n5 más largas:")
display(df.sort_values("n_words", ascending=False).head(5)[["id","origen","n_words"]])


Resumen por origen (n_words):
        count        mean         std    min     25%    50%     75%     max
origen                                                                     
humano    4.0  698.750000  287.729474  401.0  496.25  679.5  882.00  1035.0
llm     144.0  445.076389  117.917186   69.0  394.25  477.5  518.25   772.0

5 más cortas:


,id,origen,n_words
8,gemma_3_4b_f1_c05,llm,69
128,mistral_f3_c05,llm,133
140,mistral_f4_c05,llm,149
20,gemma_3_4b_f2_c05,llm,160
116,mistral_f2_c05,llm,165



5 más largas:


,id,origen,n_words
0,gold_f1_Dortmund__Real_Madrid,humano,1035
2,gold_f3_Liverpool___Milán,humano,831
113,mistral_f2_c02,llm,772
65,qwen_3_4b_2507_f2_c02,llm,683
7,gemma_3_4b_f1_c04,llm,653


In [14]:
def obtener_lemas(texto):
    doc = nlp(texto)
    # solo tokens alfabéticos, en minúsculas
    lemas = [t.lemma_.lower() for t in doc if t.is_alpha]
    return lemas

# generar columna con lemas
df["lemas"] = df["texto"].apply(obtener_lemas)

# comprobación rápida
print("Ejemplo de lemas (primer texto):")
print(df.iloc[0]["lemas"][:30])

# comprobar longitud de lemas vs n_words
df[["n_words"]].head()


Ejemplo de lemas (primer texto):
['carvajal', 'servir', 'el', 'al', 'rey', 'de', 'europa', 'uno', 'cabezazo', 'soberbio', 'del', 'canterano', 'inclín', 'uno', 'final', 'igualadísimo', 'con', 'uno', 'dortmund', 'pujante', 'frenado', 'por', 'courtois', 'en', 'el', 'primero', 'tiempo', 'vinicius', 'cerrar', 'el']


,n_words
0,1035
1,528
2,831
3,401
4,534


In [15]:
def obtener_lemas_content(texto):
    doc = nlp(texto)
    content_pos = {"NOUN", "VERB", "ADJ", "ADV"}
    lemas = [
        t.lemma_.lower()
        for t in doc
        if t.is_alpha and (t.pos_ in content_pos) and (not t.is_stop)
    ]
    return lemas

df["lemas_content"] = df["texto"].apply(obtener_lemas_content)

print("Ejemplo lemas_content (primer texto):")
print(df.iloc[0]["lemas_content"][:30])

print("\nTamaños (primer texto):")
print("n_words:", df.iloc[0]["n_words"])
print("len(lemas):", len(df.iloc[0]["lemas"]))
print("len(lemas_content):", len(df.iloc[0]["lemas_content"]))


Ejemplo lemas_content (primer texto):
['servir', 'cabezazo', 'soberbio', 'canterano', 'inclín', 'igualadísimo', 'pujante', 'frenado', 'tiempo', 'vinicius', 'cerrar', 'victoria', 'tramo', 'competición', 'querer', 'dominar', 'equipo', 'blanco', 'juega', 'gana', 'sufrir', 'inferior', 'tiempo', 'sorprendente', 'golpear', 'inesperado', 'balón', 'aéreo', 'córner', 'rematar']

Tamaños (primer texto):
n_words: 1035
len(lemas): 1035
len(lemas_content): 352


# Calculo la priemra métrica: MTLD (primero con content y después de manera general)

In [16]:
from lexical_diversity import lex_div as ld
import numpy as np

def calcular_mtld(lemas):
    # lexical-diversity espera lista de tokens
    if len(lemas) < 100:
        return np.nan
    return ld.mtld(lemas)

# MTLD sobre palabras de contenido
df["mtld_content"] = df.apply(
    lambda row: calcular_mtld(row["lemas_content"])
    if row["n_words"] >= 100 else np.nan,
    axis=1
)

# resumen por origen
print("MTLD (contenido) — resumen por origen:")
print(df.groupby("origen")["mtld_content"].describe())


MTLD (contenido) — resumen por origen:
        count        mean         std         min         25%         50%  \
origen                                                                      
humano    4.0  309.081360  128.128883  198.603243  213.490811  281.236989   
llm     126.0  227.974829   95.363340   55.011335  150.128822  222.668552   

               75%         max  
origen                          
humano  376.827539  475.248219  
llm     281.551811  436.813333  


Ahora lo hacemos por modelo y configuración

In [17]:
import re

def extraer_modelo(id_texto):
    if id_texto.startswith("gemma"):
        return "gemma"
    if id_texto.startswith("qwen"):
        return "qwen"
    if id_texto.startswith("mistral"):
        return "mistral"
    return "humano"

def extraer_config(id_texto):
    m = re.search(r"_c(\d+)", id_texto)
    return int(m.group(1)) if m else None

df["modelo"] = df["id"].apply(extraer_modelo)
df["config"] = df["id"].apply(extraer_config)


In [18]:
gold_mean = df.loc[df["origen"]=="humano", "mtld_content"].mean()
gold_std  = df.loc[df["origen"]=="humano", "mtld_content"].std()

print("Gold standard humano (MTLD_content):")
print("Media:", round(gold_mean,2))
print("STD:", round(gold_std,2))


Gold standard humano (MTLD_content):
Media: 309.08
STD: 128.13


In [19]:
mtld_por_config = (
    df[df["origen"]=="llm"]
    .groupby("config")["mtld_content"]
    .agg(["count","mean","std"])
    .sort_values("mean", ascending=False)
)

mtld_por_config


,count,mean,std
config,,,
4.0,12,294.020129,114.459348
7.0,12,249.545249,92.122857
8.0,12,249.538412,107.068258
3.0,12,247.546995,78.762552
12.0,12,224.960907,84.304880
9.0,11,217.461383,110.796293
2.0,12,212.832767,85.939984
1.0,12,210.645041,99.152758
10.0,8,208.997135,127.218275


Vemos la distancia con texto humano z_dist

In [20]:
mtld_por_config["z_dist"] = (
    (mtld_por_config["mean"] - gold_mean).abs() / gold_std
)

mtld_por_config.sort_values("z_dist")


,count,mean,std,z_dist
config,,,,
4.0,12,294.020129,114.459348,0.117548
7.0,12,249.545249,92.122857,0.464658
8.0,12,249.538412,107.068258,0.464711
3.0,12,247.546995,78.762552,0.480254
12.0,12,224.960907,84.304880,0.656530
9.0,11,217.461383,110.796293,0.715061
2.0,12,212.832767,85.939984,0.751186
1.0,12,210.645041,99.152758,0.768260
10.0,8,208.997135,127.218275,0.781122


In [21]:
mtld_config_modelo = (
    df[df["origen"]=="llm"]
    .groupby(["config", "modelo"])["mtld_content"]
    .agg(["count","mean","std"])
    .reset_index()
    .sort_values(["config","mean"], ascending=[True, False])
)

mtld_config_modelo


,config,modelo,count,mean,std
1,1.0,mistral,4,270.456765,114.533575
0,1.0,gemma,4,253.214129,38.017180
2,1.0,qwen,4,108.264230,17.968741
4,2.0,mistral,4,286.025330,35.895671
3,2.0,gemma,4,240.847401,42.666115
5,2.0,qwen,4,111.625569,46.102524
7,3.0,mistral,4,297.073839,46.279706
6,3.0,gemma,4,290.367840,52.908275
8,3.0,qwen,4,155.199306,26.814988
10,4.0,mistral,4,392.169583,59.368064


# Métricas MTLD general, no solamente palabras de "content"

In [22]:
import numpy as np
from lexical_diversity import lex_div as ld

def calcular_mtld_general(lemas):
    if len(lemas) < 100:
        return np.nan
    return ld.mtld(lemas)

df["mtld_general"] = df.apply(
    lambda row: calcular_mtld_general(row["lemas"])
    if row["n_words"] >= 100 else np.nan,
    axis=1
)

print("MTLD (general) — resumen por origen:")
print(df.groupby("origen")["mtld_general"].describe())


MTLD (general) — resumen por origen:
        count       mean        std       min        25%        50%  \
origen                                                                
humano    4.0  60.982014   3.567366  55.75372  60.349655  62.225633   
llm     143.0  61.136762  14.676085  26.65396  48.052601  64.776491   

              75%         max  
origen                         
humano  62.857992   63.723070  
llm     71.530962  104.971606  


In [23]:
# por configuración
mtldg_por_config = (
    df[df["origen"]=="llm"]
    .groupby("config")["mtld_general"]
    .agg(["count","mean","std"])
    .sort_values("mean", ascending=False)
)

mtldg_por_config


,count,mean,std
config,,,
8.0,12,64.562800,12.690850
4.0,12,63.341008,13.556150
1.0,12,62.797442,15.196395
3.0,12,62.354102,16.262435
5.0,11,62.331866,15.220322
7.0,12,61.673133,13.995278
10.0,12,61.580313,22.277704
12.0,12,61.098964,13.363642
11.0,12,59.732033,11.733289


In [24]:
#por modelo
mtldg_config_modelo = (
    df[df["origen"]=="llm"]
    .groupby(["config","modelo"])["mtld_general"]
    .agg(["count","mean","std"])
    .reset_index()
    .sort_values(["config","mean"], ascending=[True, False])
)

mtldg_config_modelo


,config,modelo,count,mean,std
0,1.0,gemma,4,74.237457,1.269285
1,1.0,mistral,4,71.114612,6.717618
2,1.0,qwen,4,43.040258,3.579566
3,2.0,gemma,4,70.267423,2.904637
4,2.0,mistral,4,65.826385,9.676389
5,2.0,qwen,4,39.307049,8.526645
6,3.0,gemma,4,74.975016,10.432937
7,3.0,mistral,4,68.942272,9.186540
8,3.0,qwen,4,43.145018,3.772581
9,4.0,gemma,4,73.804506,11.083798


# Paso a medir complejidad sintáctica: Longitud media de oración - Número medio de tokens por oración

In [25]:
import numpy as np

def metricas_sintacticas_basicas(doc):
    sent_lengths_words = []
    sent_lengths_tokens = []

    for sent in doc.sents:
        tokens = [
            t for t in sent
            if not t.is_punct and not t.is_space
        ]
        if len(tokens) > 0:
            sent_lengths_tokens.append(len(tokens))
            sent_lengths_words.append(
                len([t for t in tokens if t.is_alpha])
            )

    return {
        "n_sentences": len(sent_lengths_tokens),
        "mean_sent_len_tokens": np.mean(sent_lengths_tokens) if sent_lengths_tokens else np.nan,
        "mean_sent_len_words": np.mean(sent_lengths_words) if sent_lengths_words else np.nan
    }


In [26]:
df["doc"] = df["texto"].apply(nlp)


In [27]:
# aplico las métricas sintácticas básicas
sintacticas = df["doc"].apply(metricas_sintacticas_basicas).apply(pd.Series)

df = pd.concat([df, sintacticas], axis=1)


In [28]:
#compruebo
df[["origen", "n_sentences", "mean_sent_len_words", "mean_sent_len_tokens"]].head()


,origen,n_sentences,mean_sent_len_words,mean_sent_len_tokens
0,humano,68.0,15.220588,15.338235
1,humano,18.0,29.333333,30.277778
2,humano,46.0,18.065217,18.217391
3,humano,16.0,25.062500,25.437500
4,llm,22.0,24.272727,24.909091


In [29]:
df.groupby("origen")[[
    "n_sentences",
    "mean_sent_len_words",
    "mean_sent_len_tokens"
]].agg(["mean", "std", "min", "max"])


n_sentences                        mean_sent_len_words            \
              mean        std   min   max                mean       std   
origen                                                                    
humano   37.000000  24.792472  16.0  68.0           21.920410  6.443971   
llm      24.395833  10.888641   4.0  90.0           19.673266  4.657347   

                             mean_sent_len_tokens                       \
              min        max                 mean       std        min   
origen                                                                   
humano  15.220588  29.333333            22.317726  6.797584  15.338235   
llm      4.677778  29.166667            20.520029  4.802234   5.033333   

                   
              max  
origen             
humano  30.277778  
llm     30.764706

In [30]:
# vemos sintaxis por configuración
df[df["origen"] == "llm"].groupby("config")[[
    "n_sentences",
    "mean_sent_len_words",
    "mean_sent_len_tokens"
]].agg(["mean", "std"])


n_sentences            mean_sent_len_words            \
              mean        std                mean       std   
config                                                        
1.0      23.666667   5.210712           22.429082  5.178031   
2.0      27.250000  10.592665           21.434727  4.744973   
3.0      22.666667   4.163332           22.200645  3.375739   
4.0      28.166667   8.768055           20.449272  4.122298   
5.0       9.083333   3.088346           21.649101  4.143252   
6.0      20.250000   4.956630           21.342791  3.390691   
7.0      23.416667   6.416716           20.147565  3.204252   
8.0      24.750000   7.460502           20.078490  3.945197   
9.0      28.916667   7.668807           16.887008  2.644032   
10.0     39.416667  21.840156           11.254967  3.624359   
11.0     21.000000   4.410731           20.590283  2.082227   
12.0     24.166667   6.042777           17.615262  2.640438   

       mean_sent_len_tokens            
                       mean       std  
config                                 
1.0               23.336260  5.369420  
2.0               22.218048  4.715906  
3.0               23.119887  3.469111  
4.0               21.193289  4.172850  
5.0               22.762335  4.344004  
6.0               22.154412  3.643300  
7.0               21.001916  3.291039  
8.0               20.956374  4.092087  
9.0               17.418037  2.624304  
10.0              11.925826  3.886823  
11.0              21.647284  2.079725  
12.0              18.506683  2.799125

# Construyo el perfil estadístico humano (gold standard) calculando la media y desviación estándar de las métricas lingüísticas para usarlo como referencia de comparación.

In [31]:
metricas_finales = [
    "mtld_content",
    "mtld_general",
    "n_sentences",
    "mean_sent_len_words"
]

gold_profile = (
    df[df["origen"] == "humano"][metricas_finales]
    .agg(["mean", "std"])
    .T
)

gold_profile


,mean,std
mtld_content,309.081360,128.128883
mtld_general,60.982014,3.567366
n_sentences,37.000000,24.792472
mean_sent_len_words,21.920410,6.443971


# Paso 16.2 — Significancia estadística: compruebo si las diferencias observadas entre crónicas humanas y LLM son estadísticamente significativas.

In [32]:
from scipy.stats import mannwhitneyu


In [33]:
def test_significancia(df, columna):
    humano = df[df["origen"] == "humano"][columna].dropna()
    llm = df[df["origen"] == "llm"][columna].dropna()

    stat, p = mannwhitneyu(humano, llm, alternative="two-sided")

    return {
        "metrica": columna,
        "U": stat,
        "p_value": p,
        "humano_mean": humano.mean(),
        "llm_mean": llm.mean(),
        "humano_n": len(humano),
        "llm_n": len(llm)
    }


In [34]:
metricas = [
    "mtld_content",
    "mtld_general",
    "n_sentences",
    "mean_sent_len_words"
]

resultados_stats = pd.DataFrame(
    [test_significancia(df, m) for m in metricas]
)

resultados_stats


,metrica,U,p_value,humano_mean,llm_mean,humano_n,llm_n
0,mtld_content,348.0,0.197924,309.081360,227.974829,4,126
1,mtld_general,235.0,0.561427,60.982014,61.136762,4,143
2,n_sentences,330.5,0.619018,37.000000,24.395833,4,144
3,mean_sent_len_words,349.5,0.470722,21.920410,19.673266,4,144


Se pueden observar diferencias descriptivas entre grupos, pero no alcanzan significación estadística en esta muestra reducida, son solamente 4 muestras de humanos.

# Calculo el tamaño del efecto (r) del test Mann–Whitney para cuantificar cómo de grande es la diferencia entre crónicas humanas y LLM, independientemente de que sea o no estadísticamente significativa.

In [35]:
from scipy.stats import mannwhitneyu
import numpy as np
import pandas as pd
from math import sqrt

def mannwhitney_con_efecto(df, col):
    x = df[df["origen"]=="humano"][col].dropna()
    y = df[df["origen"]=="llm"][col].dropna()

    U, p = mannwhitneyu(x, y, alternative="two-sided")

    n1, n2 = len(x), len(y)
    mu_U = n1*n2/2
    sigma_U = sqrt(n1*n2*(n1+n2+1)/12)

    z = (U - mu_U) / sigma_U
    r = abs(z) / sqrt(n1+n2)   # tamaño del efecto tipo r

    return {
        "metrica": col,
        "U": U,
        "p_value": p,
        "z": z,
        "r_effect": r,
        "humano_mean": x.mean(),
        "llm_mean": y.mean(),
        "humano_n": n1,
        "llm_n": n2
    }

metricas = ["mtld_content","mtld_general","n_sentences","mean_sent_len_words"]

resultados_stats_efecto = pd.DataFrame([mannwhitney_con_efecto(df, m) for m in metricas])
resultados_stats_efecto


,metrica,U,p_value,z,r_effect,humano_mean,llm_mean,humano_n,llm_n
0,mtld_content,348.0,0.197924,1.294228,0.113511,309.081360,227.974829,4,126
1,mtld_general,235.0,0.561427,-0.607200,0.050081,60.982014,61.136762,4,143
2,n_sentences,330.5,0.619018,0.502545,0.041309,37.000000,24.395833,4,144
3,mean_sent_len_words,349.5,0.470722,0.727213,0.059777,21.920410,19.673266,4,144


# Mido la distancia de las crónicas al gold standard

In [36]:
# perfil humano (gold standard)
gold_profile = df[df["origen"] == "humano"][[
    "mtld_content",
    "mtld_general",
    "n_sentences",
    "mean_sent_len_words"
]].agg(["mean", "std"])

gold_profile


,mtld_content,mtld_general,n_sentences,mean_sent_len_words
mean,309.081360,60.982014,37.000000,21.920410
std,128.128883,3.567366,24.792472,6.443971


In [37]:
metricas = [
    "mtld_content",
    "mtld_general",
    "n_sentences",
    "mean_sent_len_words"
]

for m in metricas:
    mu = gold_profile.loc["mean", m]
    sd = gold_profile.loc["std", m]
    df[f"z_{m}"] = (df[m] - mu) / sd


In [38]:
for m in metricas:
    df[f"dist_{m}"] = df[f"z_{m}"].abs()


In [39]:
df["human_distance_score"] = df[
    [f"dist_{m}" for m in metricas]
].mean(axis=1)


In [40]:
df[[
    "origen",
    "modelo",
    "config",
    "human_distance_score"
]].sort_values("human_distance_score").head(10)


,origen,modelo,config,human_distance_score
7,llm,gemma,4.0,0.320418
34,llm,gemma,7.0,0.339278
129,llm,mistral,6.0,0.371602
10,llm,gemma,7.0,0.383150
2,humano,humano,NaN,0.419751
102,llm,mistral,3.0,0.449375
120,llm,mistral,9.0,0.457839
122,llm,mistral,11.0,0.484415
118,llm,mistral,7.0,0.486123
33,llm,gemma,6.0,0.510970


In [41]:
# Ranking por configuración
ranking_config = (
    df[df["origen"] == "llm"]
    .groupby("config")["human_distance_score"]
    .agg(["mean", "std"])
    .sort_values("mean")
)

ranking_config


,mean,std
config,,
11.0,1.209243,0.434957
6.0,1.209650,0.696068
4.0,1.211186,0.523428
7.0,1.217238,0.616589
12.0,1.296203,0.555956
8.0,1.299448,0.311318
9.0,1.310969,0.862451
3.0,1.367374,0.612034
2.0,1.415474,0.764236


In [42]:
# Ranking por modelo
ranking_modelo = (
    df[df["origen"] == "llm"]
    .groupby("modelo")["human_distance_score"]
    .agg(["mean", "std"])
    .sort_values("mean")
)

ranking_modelo


,mean,std
modelo,,
mistral,1.041205,0.437021
gemma,1.281244,0.760019
qwen,1.908562,0.613033


In [43]:
# Ranking por modelo y config
ranking_modelo_config = (
    df[df["origen"] == "llm"]
    .groupby(["modelo", "config"])["human_distance_score"]
    .agg(["mean", "std", "count"])
    .sort_values("mean")
)

ranking_modelo_config.head(20)


mean       std  count
modelo  config                           
mistral 9.0     0.664270  0.157474      4
        6.0     0.690547  0.251680      4
gemma   7.0     0.839994  0.762411      4
        9.0     0.895117  0.366047      4
mistral 11.0    0.951123  0.454961      4
        3.0     0.954928  0.504042      4
        12.0    0.967367  0.436289      4
        2.0     0.974638  0.310552      4
        4.0     1.026096  0.312316      4
gemma   2.0     1.027997  0.212287      4
        12.0    1.055382  0.166234      4
        11.0    1.088125  0.389662      4
mistral 8.0     1.101080  0.464397      4
gemma   4.0     1.140291  0.856806      4
mistral 10.0    1.153943  0.378659      4
gemma   6.0     1.157590  0.477382      4
mistral 1.0     1.198963  0.403392      4
        7.0     1.223619  0.578364      4
gemma   1.0     1.296060  0.062363      4
        3.0     1.300384  0.678543      4

In [44]:
# Top 5
ranking_modelo_config.head(5)



mean       std  count
modelo  config                           
mistral 9.0     0.664270  0.157474      4
        6.0     0.690547  0.251680      4
gemma   7.0     0.839994  0.762411      4
        9.0     0.895117  0.366047      4
mistral 11.0    0.951123  0.454961      4

In [45]:
# el peor
ranking_modelo_config.tail(5)


mean       std  count
modelo config                           
qwen   1.0     2.034578  0.289529      4
       2.0     2.243787  0.791158      4
       9.0     2.373521  0.524401      4
gemma  10.0    2.412194  1.817092      4
qwen   10.0    2.943843  0.841087      4

# Ahora compruebo la significancia a nivel modelo/config.

In [46]:
from scipy.stats import kruskal
import itertools
from scipy.stats import mannwhitneyu

# Kruskal-Wallis: ¿hay diferencias entre modelos en human_distance_score?
g = df[df["origen"] == "llm"]
groups = [g[g["modelo"] == m]["human_distance_score"].dropna() for m in g["modelo"].unique()]
H, p = kruskal(*groups)
H, p


(np.float64(49.37315613026823), np.float64(1.9000077361089413e-11))

In [47]:
modelos = g["modelo"].unique()
posthoc = []

for a, b in itertools.combinations(modelos, 2):
    xa = g[g["modelo"] == a]["human_distance_score"].dropna()
    xb = g[g["modelo"] == b]["human_distance_score"].dropna()
    U, pval = mannwhitneyu(xa, xb, alternative="two-sided")
    posthoc.append((a, b, U, pval))

posthoc


[('gemma', 'qwen', np.float64(436.0), np.float64(1.5805584973395988e-07)),
 ('gemma', 'mistral', np.float64(1377.0), np.float64(0.09995890576637419)),
 ('qwen', 'mistral', np.float64(2053.0), np.float64(4.152527530685214e-11))]

Significancia entre configs (solo LLM)

In [48]:
configs = sorted(g["config"].dropna().unique())
groups_c = [g[g["config"] == c]["human_distance_score"].dropna() for c in configs]
Hc, pc = kruskal(*groups_c)
Hc, pc


(np.float64(12.688984674329504), np.float64(0.3141382650952812))